<a href="https://colab.research.google.com/github/clecioSantos/goTranslate/blob/main/projeto_imers%C3%A3o_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [522]:
%pip -q install google-genai

In [523]:
################################################################################
# Configura a API Key do Google Gemini
################################################################################

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [524]:
################################################################################
# Configura o cliente da SDK do Gemini
################################################################################

from google import genai

In [525]:
################################################################################
# Instalar Framework ADK de agentes do Google
################################################################################

!pip install -q google-adk

In [526]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [527]:
################################################################################
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
################################################################################
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response  = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [528]:
################################################################################
# --- Agente 1: Busca frases para serem traduzidas
################################################################################
def agente_buscador(tema, nivel):

  buscador = Agent(
      name        = "agente_buscador",
      model       = "gemini-2.0-flash",
      tools       = [google_search],
      instruction = """Você é um assitente de pesquisa e a sua tarefa é usar a ferramenta de busca do google (google_search)
                       para buscar exemplos de frases em ingles que ajudem a ensinar o tema pedido para o nivel de conhecimento informado.
                    """,
      description = "Agente que busca exercicios de ingles"
  )

  entrada_do_agente_buscador = f"Tema: {tema} \nnivel de ingles: {nivel}"
  # Executa o agente
  lancamentos_buscados = call_agent(buscador, entrada_do_agente_buscador)
  return lancamentos_buscados

In [529]:
################################################################################
# --- Agente 2: Criador de frases
################################################################################
def agente_planejador(tema, exercicios_sugeridos):
    planejador = Agent(
        name        = "agente_planejador",
        model       = "gemini-2.0-flash",
        instruction = """ Você é um planejador de aula de ingles e deve utilizar os exercicios sugeridos de base para criaruma frase sobre o mesmo tema,
                          você deve criar uma frase para ser traduzida.
                      """,
        description = "Agente que planeja aulas",
        tools       = [google_search]
    )

    entrada_do_agente_planejador = f"Tópico:{tema}\nExercicios sugeridos: {exercicios_sugeridos}"
    # Executa o agente
    plano_de_aula = call_agent(planejador, entrada_do_agente_planejador)
    return plano_de_aula

In [530]:
################################################################################
# --- Agente 3: Professor
################################################################################
def agente_professor(topico, plano_de_aula):
    redator = Agent(
        name="agente_redator",
        model="gemini-2.0-flash",
        instruction="""
            Você é um professor de ingles e deve fazer uma das perguntas do plano de aula de forma sucinta e em ingles,
            apenas indicando que o aluno deve traduzi-la e removendo qualquer parte que seja em portugues
            """,
        description="Agente redator de posts engajadores para Instagram"
    )
    entrada_do_agente_redator = f"Tópico: {topico}\nPlano de aula: {plano_de_aula}"
    # Executa o agente
    aula = call_agent(redator, entrada_do_agente_redator)
    return aula

In [531]:
################################################################################
# --- Agente 4: Revisor
################################################################################
def agente_revisor(plano_de_aula, resposta, nivel):
    revisor = Agent(
        name="agente_revisor",
        model="gemini-2.0-flash",
        description="Agente revisor de post para redes sociais.",
        instruction="""Você é um revisor de tradução, serão passadas 3 informações para você:
            1- plano de aula: indica a frase em ingles que o usuario deve traduzir;
            2- resposta: indica a resposta do cliente
            3- nivel: o nivel de ingles do usuario
            você deve considerar esses pontos e retornar se o usuario acertou a tradução,
            após corrigir retorne pontos positivos e negativos da resposta dada,
            mas sempre tentando insentivar o aluno a seguir estudando e enfatizando seus avanços.
            sempre responda em portugues e em texto livre, sem markdown.
            responda como se já estivesse em uma conversa com o usuário, portanto nã ose apresente e nem dê oi
            """
    )
    entrada_do_agente_revisor = f"Plano de aula: {plano_de_aula}\nresposta: {resposta}\nnivel:{nivel}"
    # Executa o agente
    prova_revisada = call_agent(revisor, entrada_do_agente_revisor)
    return prova_revisada

In [532]:
################################################################################
# --- Agente 5: Agente de continuidade
################################################################################
def agente_de_continuidade(resposta):
    revisor = Agent(
        name="agente_de_continuidade",
        model="gemini-2.0-flash",
        instruction="""
            Você deve interpletar a resposta do usuario e entender se ele pretende continuar no sistema ou não,
            e responder 1 para sim e 0 para não.
            A pergunta feita ao usuario foi: "Deseja continuar aprendendo?"
            """,
        description="Agente revisor de post para redes sociais."
    )
    entrada_do_agente_revisor = f"resposta: {resposta}"
    # Executa o agente
    prova_revisada = call_agent(revisor, entrada_do_agente_revisor)
    return prova_revisada

In [534]:
from enum import Enum

class Estados(Enum):
    TEMA        = 1
    NIVEL       = 2
    PROCESSANDO = 3
    RESPOSTA    = 4
    CONTINUAR   = 5
    FIM         = 6



In [535]:
################################################################################
# --- Processa o fim do chat
################################################################################
def process_fim():
    estado_atual = Estados.FIM
    return "Ok, até a próxima"

In [558]:
aula = ''
################################################################################
# --- Inicia o trabalho dos agentes, criando a frase que deverá ser traduzida
################################################################################
def start_agent():
    global estado_atual, tema, nivel, plano_de_aula, aula

    add_message_to_chat('Um instante..', 'chatbot')
    estado_atual         = Estados.PROCESSANDO
    exercicios_sugeridos = agente_buscador(tema, nivel)
    plano_de_aula        = agente_planejador(tema, exercicios_sugeridos)
    aula                 = agente_professor(tema, plano_de_aula)
    add_message_to_chat(aula, 'chatbot')
    estado_atual         = Estados.RESPOSTA

In [556]:
tema = ''
################################################################################
# --- Processa a leitura do tema
################################################################################
def process_tema(message):
    if message:
        global estado_atual, tema, nivel
        tema         = message
        estado_atual = Estados.NIVEL
        add_message_to_chat(f'Certo, vamos aprender sobre {tema}', 'chatbot')
        return "Para que eu possa personalizar a sua experiencia, me diga o seu nivel de ingês"
    else:
        return process_fim()

In [557]:
nivel = ''
################################################################################
# --- Processa a leitura do nivel
################################################################################
def process_nivel(message):
    global estado_atual, tema, nivel
    if message:
        nivel = message
        add_message_to_chat(f'ok, vamos no nivel {nivel}', 'chatbot')
        estado_atual = Estados.PROCESSANDO
        start_agent()
        return ""
    else:
        return process_fim()

In [559]:
################################################################################
# --- Processa a resposta do usuario
################################################################################
def process_resposta(message):
    global estado_atual, tema, nivel, aula
    resultado = agente_revisor(aula, message, nivel)
    add_message_to_chat(resultado, 'chatbot')
    estado_atual = Estados.CONTINUAR
    return "Deseja continuar aprendendo?"

In [553]:
################################################################################
# --- Verifica se o cliente quer continuar estudando
################################################################################
def process_continuar(message):
    global estado_atual
    continuar = agente_de_continuidade(message)
    if '1' in continuar:
        estado_atual = Estados.PROCESSANDO
        start_agent()
        return ''
    else:
        return process_fim()

In [541]:
################################################################################
# --- Processa a mensagem do usuario
################################################################################
def process_user_message(message):
    global estado_atual
    if estado_atual == Estados.TEMA:
        return process_tema(message)
    elif estado_atual == Estados.NIVEL:
        return process_nivel(message)
    elif estado_atual == Estados.RESPOSTA:
        return process_resposta(message)
    elif estado_atual == Estados.CONTINUAR:
        return process_continuar(message)


In [542]:
################################################################################
# --- Processa o click no botão enviar
################################################################################
def process_input(b):
    global estado_atual
    input_text = input_field.value

    if input_text:
        if estado_atual == Estados.PROCESSANDO:
            add_message_to_chat("Estou preparando a frase. Por favor, aguarde.", 'chatbot')
        elif estado_atual != Estados.FIM:
            add_message_to_chat(input_text, 'user')

            # --- Processa a mensagem do usuário usando a função dedicada ---
            message_boot = process_user_message(input_text)
            if message_boot:
                add_message_to_chat(message_boot, 'chatbot')

            tema = input_text
            # Limpa o campo de entrada após processar
            input_field.value = ''
    else:
        # Opcional: exibir uma mensagem se o campo estiver vazio
        add_message_to_chat("Por favor, digite algo.", 'chatbot')

# HTML

In [583]:
################################################################################
# --- Cria a aba do chat
################################################################################
import ipywidgets as widgets
from IPython.display import display, HTML, clear_output
from ipywidgets import Output, VBox, HBox # Importa VBox e HBox
import markdown

# CSS para estilizar a saída dentro do widget Output
# Este CSS será injetado no HTML gerado para cada mensagem.
css_style = """
<style>
  .styled-input-output {
    background-color: #eef;
    border: 1px solid #ccf;
    /* Diminui o padding e a margem inferior para reduzir a altura de cada mensagem */
    padding: 10px 10px; /* Ajusta o padding vertical e horizontal  */
    margin-bottom: 5px; /* Mantém a margem entre as mensagens */
    border-radius: 5px;
    font-family: sans-serif; /* Fonte mais comum para interfaces */
    white-space: pre-wrap; /* Preserva quebras de linha e espaços */
    word-wrap: break-word; /* Quebra palavras longas */
    color: #000; /* Cor do texto principal */
    max-width: 80%; /* Limita a largura máxima do bloco */
    margin-right: auto; /* Alinha o bloco à esquerda */
    line-height: 1
    margin: 0
    padding: 0
  }
  /* Opcional: Estilo para simular mensagens do usuário alinhadas à direita */
  .styled-input-output.user-input {
      background-color: #d4edda; /* Fundo diferente */
      border-color: #c3e6cb;
      margin-left: auto; /* Alinha o bloco à direita */
      margin-right: 0;
  }
  /* Estilo para o contêiner da área de saída para dar uma altura mínima */
  /* overflow-y: auto; já adiciona o scroll vertical quando necessário */
  .output {
      border: 1px solid #ccc; /* Adiciona uma borda para visualizar a área */
      min-height: 800px; /* Altura mínima revertida para 400px */
      max-width: 1850px; /* Largura máxima do bloco de chat */
      min-width: 1850px; /* Largura máxima do bloco de chat */
      max-height: 700px; /* Altura máxima revertida para 400px */
      overflow-y: auto; /* Adiciona scroll se o conteúdo exceder a altura */
      padding: 10px;
      margin-bottom: 10px; /* Espaço entre a área de chat e o input */
      border-radius: 5px;
      display: flex; /* Usa flexbox para alinhar mensagens */
      flex-direction: column; /* Empilha as mensagens verticalmente */
  }
</style>
"""

# Cria os widgets
# Campo de texto para a entrada do usuário
input_field = widgets.Text(
    value='',
    placeholder='Digite algo aqui...',
    description='Entrada:',
    disabled=False,
    # Ajusta a largura do campo de entrada para deixar espaço para o botão
    # A largura total do HBox (input + button) deve ser aproximadamente 450px
    layout=widgets.Layout(width='360px') # Experimente este valor
)

# Botão para enviar a entrada
send_button = widgets.Button(
    description='Enviar', # Texto do botão alterado para "Enviar"
    disabled=False,
    button_style='primary', # Estilo visual do botão alterado para azul ('primary')
    tooltip='Clique para enviar sua mensagem', # Tooltip atualizado
    icon='play', # Ícone do Font Awesome
    # Define a largura do botão para o diminuir em cerca de 20%
    layout=widgets.Layout(width='72px') # Largura estimada para 20% menor que ~90px
)

# Área de saída para exibir o texto estilizado
# Adicionamos a classe 'output' para aplicar o estilo definido no CSS
# O layout do widget Output já inclui min_height e overflow_y='auto'
output_area = Output(layout=widgets.Layout(width='100%', border='1px solid #ccc', min_height='400px', max_width='450px', max_height='400px', overflow_y='auto', padding='10px', margin_bottom='10px'))
# Nota: O estilo 'output' no CSS também ajuda, mas definir alguns layouts básicos aqui
# garante que o widget Output tenha as dimensões corretas antes mesmo do CSS ser aplicado.
# Adicionei max_height aqui também para um controle mais preciso.


# --- Função para adicionar uma mensagem ao chat ---
def add_message_to_chat(message, sender):
    """
    Adiciona uma mensagem à área de chat com o estilo apropriado e tenta rolar para o final.

    Args:
        message (str): O texto da mensagem a ser exibida.
        sender (str): O remetente da mensagem ('user' ou 'chatbot').
    """
    # Determina a classe CSS com base no remetente
    message_class = "styled-input-output"
    if sender == 'user':
        message_class += " user-input" # Adiciona a classe para mensagens do usuário

    # Cria o HTML para a mensagem
    html_content = markdown.markdown(message)
    message_html = f"""
    <div class="{message_class}">
        {html_content}
    </div>
    """

    # Script JavaScript para rolar a área de saída para o final
    # Usamos scrollIntoView no último elemento de mensagem dentro da área de saída
    scroll_script = """
    <script>
        // Aumentamos o delay para dar mais tempo para o DOM atualizar
        // Mantemos um pequeno delay, mas a abordagem de scrollIntoView no elemento específico é mais robusta
        setTimeout(function() {
            // Encontra o contêiner de scroll (o elemento div principal do widget Output)
            // Procuramos pelo último elemento com a classe 'output' que foi renderizado
            const outputElements = document.querySelectorAll('.output');
            if (outputElements.length > 0) {
                const scrollableContainer = outputElements[outputElements.length - 1];
                // Rola para o final do conteúdo imediatamente
                scrollableContainer.scrollTop = scrollableContainer.scrollHeight;
            }
        }, 200); // Delay de 200ms
    </script>
    """

    # Combina o HTML da mensagem e o script de scroll
    combined_html = message_html + scroll_script

    # Exibe o HTML combinado na área de saída
    with output_area:
        # Inclui o CSS aqui também (redundante mas seguro)
        display(HTML(css_style + combined_html))




# Execução do chat

In [584]:
################################################################################
# --- Executa o chat
################################################################################
output_area = Output(layout=widgets.Layout(width='100%', border='1px solid #ccc', min_height='850px', max_width='850px', min_width='850px', max_height='850px', overflow_y='auto', padding='10px', margin_bottom='10px'))

estado_atual = Estados.TEMA
# Cria o contêiner para o campo de entrada e o botão
input_container = HBox([input_field, send_button])

# Cria o contêiner vertical principal que irá centralizar o chat
chat_container = VBox([output_area, input_container],
                      layout=widgets.Layout(align_items='center')) # Centraliza os itens horizontalmente
# Vincula a função ao evento de clique do botão
send_button.on_click(process_input)

# Exibe os widgets na interface do Colab na ordem desejada (saída primeiro, depois input)
display(output_area, widgets.HBox([input_field, send_button]))

# --- Adiciona a mensagem de boas-vindas ao iniciar o chat ---
add_message_to_chat("Olá! Você iniciou o GoTranslate.", 'chatbot')

add_message_to_chat("Por favor, diga o Tema que pretende estudar: ", 'chatbot')


Output(layout=Layout(border='1px solid #ccc', max_height='850px', max_width='850px', min_height='850px', min_w…